In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
from tqdm.notebook import tqdm
from selenium.webdriver.chrome.options import Options
import re
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

In [ ]:
data = pd.read_csv('C:/Users/dudtk/Downloads/educateAD_1.csv' , sep=",", encoding='utf8')
urlList = data['광고동영상url']
header = "https://www.youtube.com/watch?v="
count = len(urlList)
options = Options()
options.add_argument('--start-maximized')
options.add_argument('--headless')
comment_list = []
foreign_list = []

def isKorean(s):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
    result = hangul.sub('', s) # 한글과 띄어쓰기를 제외한 모든 부분을 제거
    if len(result) >= 1:
        return True
    else:
        return False

In [ ]:
for i in tqdm(range(count)):
    geturl = header + urlList[i]
    comment = ''
    foreign = 0.0
    koreanCount = 0.0
    try:
        driver = webdriver.Chrome(executable_path='C:/chromedriver', chrome_options=options ) 
        driver.get(url=geturl)
        time.sleep(1)
        prev_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0,document.documentElement.scrollHeight)")

            time.sleep(1)

            curr_height = driver.execute_script("return document.documentElement.scrollHeight")

            if(curr_height == prev_height):
                break
            else:
                prev_height = driver.execute_script("return document.documentElement.scrollHeight")
        time.sleep(1)
        commentList = driver.find_elements_by_tag_name('ytd-comment-thread-renderer > ytd-comment-renderer > \
                                                       div > div > ytd-expander > div > yt-formatted-string')
        time.sleep(1)
        cnt = int(len(commentList)/2)
        for j in tqdm(range(len(commentList))):
            if len(commentList[j].text) >= 1 :
                comment += commentList[j].text
                if(isKorean(commentList[j].text)):
                    koreanCount += 1
        foreign = float((cnt-koreanCount)/cnt)
        print(foreign)
    except:
        comment = ''
        foreign = 0
        print(i,"번째 행 댓글 막혔거나 영상 삭제됬거나 어쨋든 잘 안된듯..")
    comment_list.append(comment)
    foreign_list.append(foreign)
    driver.quit()
data['외국인 비율'] = foreign_list
data['모든 댓글'] = comment_list
data.to_csv('C:/Users/dudtk/Downloads/educateAD_1_22.csv', encoding='utf-8-sig')